# 06/30/20 - This notebook readdresses the question of other surveys' overlap in the GAMA regions, using Lambdar instead of MagPhys.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy import constants as const
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
# import ephem
from astropy.cosmology import FlatLambdaCDM

In [4]:
# create datapaths for files and figures
csv_path = '/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/'
png_path = '/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Visuals/PNGs/'
pdf_path = '/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Visuals/Final_PDFs_for_Paper/lambdar_revisions/'

In [7]:
# Reading in Data
spec = pd.read_csv(f'{csv_path}spec_lambdar_revisions_063020.csv')
mac = pd.read_csv(f'{csv_path}mac_lambdar_revisions_063020.csv')
zoo = pd.read_csv(f'{csv_path}zoo_lambdar_revisions_063020.csv')

In [3]:
# Opening GAMA data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/SpecAll.fits')  # open a FITS file
hdul.verify('fix')
gama = hdul[1].data  # assume the first extension is a table
print(gama.columns)

gama_id = gama['CATAID']
gama_ra = gama['RA']
gama_dec = gama['DEC']

gama_list = list(zip(gama_id,gama_ra,gama_dec))

gama_data = pd.DataFrame(gama_list, columns = ['GAMA_ID','RA','DEC'])

ColDefs(
    name = 'SPECID'; format = '33A'
    name = 'SURVEY'; format = '9A'
    name = 'SURVEY_CODE'; format = 'I'; null = -32768
    name = 'RA'; format = 'D'; unit = 'deg'
    name = 'DEC'; format = 'D'; unit = 'deg'
    name = 'WMIN'; format = 'E'; unit = 'A'
    name = 'WMAX'; format = 'D'; unit = 'A'
    name = 'Z'; format = 'E'
    name = 'NQ'; format = 'I'; null = -32768
    name = 'PROB'; format = 'E'
    name = 'FILENAME'; format = '67A'
    name = 'URL'; format = '88A'
    name = 'URL_IMG'; format = '92A'
    name = 'CATAID'; format = 'J'; null = -2147483648
    name = 'GAMA_NAME'; format = '23A'
    name = 'IC_FLAG'; format = 'I'; null = -32768
    name = 'DIST'; format = 'E'; unit = 'arcsec'
    name = 'IS_SBEST'; format = 'L'
    name = 'IS_BEST'; format = 'L'
)


# SLACS
### SDSS LRG

In [16]:
SLACS_data = pd.read_csv(f'{csv_path}slacs_Auger2009.csv')
#slacs_gama = SLACS_data[SLACS_data.RA_deg.notnull()]
#slacs_gama = slacs_gama.reset_index()
#slacs_gama = slacs_gama.drop(columns = ['index'])
#slacs_gama = slacs_gama.rename(columns = {'Name' : 'SDSS_ID'})
SLACS_data.columns

Index(['Unnamed: 0', 'Name', 'zlens', 'zsrc', 'rEin(kpc)', 'log_MEin_M_sun',
       'fChab\nstellarEin', 'f∗Ein', 'stellarEinfstellareff',
       'log_Mstellar_M_sun_Chab', 'error_of_mass_chab',
       'log_Mstellar_M_sun__Salpeter', 'error_of_mass_sal', 'MB', 'MV', 'MB0',
       'MV0', 'stellar_mass_chabrier', 'theta_e_chabrier'],
      dtype='object')

In [75]:
slacs_gama

,SDSS_ID,RA,Dec,RA_deg,DEC,z lens,z src
0,SDSSJ0912+0029,09h12m05.31s,00 ◦ 29m01.2s,138.022125,0.483667,0.164,0.324
1,SDSSJ1143−0144,11h43m29.64s,-01 ◦ 44m30.0s,175.873500,-1.741667,0.106,0.402
2,SDSSJ1436−0000,14h36m27.54s,-00 ◦ 00m29.2s,219.114750,0.008111,0.285,0.805
3,SDSSJ1451−0239,14h51m28.19s,-02 ◦ 39m36.4s,222.867500,-2.660111,0.125,0.520


In [44]:
slacs_in_gama = pd.DataFrame(columns = ['GAMA_ID', 'SDSS_ID', 'RA', 'DEC'], index = range(len(slacs_gama)))
for i in range(len(slacs_gama.DEC)):
    Rlim = 2./3600.
    for j in range(len(gama_data)):
        R = np.sqrt(np.square(slacs_gama.DEC[i]-gama_data.DEC[j])+np.square(slacs_gama.RA_deg[i]-gama_data.RA[j]))
        if R <= Rlim:
            Rlim = R
            slacs_in_gama.GAMA_ID[i] = gama_data.GAMA_ID[j]
            slacs_in_gama.SDSS_ID[i] = slacs_gama.SDSS_ID[i]
            slacs_in_gama.RA[i] = gama_data.RA[j]
            slacs_in_gama.DEC[i] = gama_data.DEC[j]
            print(slacs_in_gama)

  GAMA_ID         SDSS_ID       RA      DEC
0  216398  SDSSJ0912+0029  138.022  0.48366
1     NaN             NaN      NaN      NaN
2     NaN             NaN      NaN      NaN
3     NaN             NaN      NaN      NaN
  GAMA_ID         SDSS_ID       RA      DEC
0  216398  SDSSJ0912+0029  138.022  0.48366
1  136604  SDSSJ1143−0144  175.873 -1.74167
2     NaN             NaN      NaN      NaN
3     NaN             NaN      NaN      NaN


In [131]:
#slacs = pd.merge(slacs_gama, slacs_in_gama, how = 'left', on = 'SDSS_ID')
#slacs = slacs.drop(columns = ['RA_x','Dec','RA_deg','DEC_x'])
#slacs = slacs.rename(columns = {'RA_y':'RA', 'DEC_y':'DEC'})
#slacs.to_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/slacs_gama.csv')
slacs

,SDSS_ID,z lens,z src,GAMA_ID,RA,DEC
0,SDSSJ0912+0029,0.164,0.324,216398,138.022,0.48366
1,SDSSJ1143−0144,0.106,0.402,136604,175.873,-1.74167
2,SDSSJ1436−0000,0.285,0.805,NaN,NaN,NaN
3,SDSSJ1451−0239,0.125,0.520,NaN,NaN,NaN


# SLACS selected two candidates that have GAMA IDs. G136604 was identified by both Mac and Zoo. 216398 was not identified in the GAMA/KiDS studies.

# S4TM

In [110]:
s4tm_data = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Publications/S4TM_data.csv')
s4tm_radec = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Publications/S4TM_radec.csv')

In [113]:
s4tm_all = pd.merge(s4tm_radec, s4tm_data, how = 'left', on = 'Plate-MJD-Fiber')
s4tm_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 105
Data columns (total 16 columns):
ra                  106 non-null float64
dec                 106 non-null float64
plate               106 non-null int64
mjd                 106 non-null int64
fiberid             106 non-null int64
Plate-MJD-Fiber     106 non-null object
Target              106 non-null object
z L                 106 non-null float64
z S                 106 non-null float64
σSDSS (km s −1)     106 non-null object
I 814 (mag)         106 non-null float64
ΔI 814 (mag)        106 non-null float64
Reff (arcsec)       106 non-null float64
q                   106 non-null float64
P.A. (deg)          106 non-null float64
Classification      106 non-null object
dtypes: float64(9), int64(3), object(4)
memory usage: 14.1+ KB


In [115]:
s4tm_in_G09 = s4tm_all[(s4tm_all.ra >= 129.0) & (s4tm_all.ra <= 141.0) & (s4tm_all.dec >= -2) & (s4tm_all.dec <= 3)]
s4tm_in_G12 = s4tm_all[(s4tm_all.ra >= 174.0) & (s4tm_all.ra <= 186.0) & (s4tm_all.dec >= -3) & (s4tm_all.dec <= 2)]                   
s4tm_in_G15 = s4tm_all[(s4tm_all.ra >= 211.5) & (s4tm_all.ra <= 223.5) & (s4tm_all.dec >= -2) & (s4tm_all.dec <= 3)]

In [120]:
s4tm_in_G15 = s4tm_in_G15.rename(columns = {'ra':'RA','dec':'DEC'})
s4tm_in_G15

,RA,DEC,plate,mjd,fiberid,Plate-MJD-Fiber,Target,z L,z S,σSDSS (km s −1),I 814 (mag),ΔI 814 (mag),Reff (arcsec),q,P.A. (deg),Classification
42,215.57813,2.054526,534,51997,481,0534-51997-481,SDSSJ1422+0203,0.1104,0.5176,172±9,16.39,0.07,2.05,0.72,175.0,E-S-C


In [133]:
#s4tm_gama = pd.merge(s4tm_in_G15, gama_data, how = 'left', on = 'RA')
#s4tm_gama = s4tm_gama.drop(s4tm_gama.index[1])
s4tm_gama.columns

Index(['RA', 'DEC_x', 'plate', 'mjd', 'fiberid', 'Plate-MJD-Fiber', 'Target',
       'z L', 'z S', 'σSDSS (km s −1) ', 'I 814 (mag) ', 'ΔI 814 (mag) ',
       'Reff (arcsec) ', 'q', 'P.A. (deg) ', 'Classification', 'GAMA_ID',
       'DEC_y'],
      dtype='object')

In [263]:
#s4tm_gama = s4tm_gama[['Target', 'GAMA_ID', 'RA', 'DEC_x', 'plate', 'mjd', 'fiberid', 'Plate-MJD-Fiber', 
      # 'z L', 'z S', 'σSDSS (km s −1) ', 'I 814 (mag) ', 'ΔI 814 (mag) ',
       #'Reff (arcsec) ', 'q', 'P.A. (deg) ', 'Classification', 
       #'DEC_y']]
#s4tm_gama.to_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/s4tm_gama.csv')
s4tm_gama

,Target,GAMA_ID,RA,DEC_x,plate,mjd,fiberid,Plate-MJD-Fiber,z L,z S,σSDSS (km s −1),I 814 (mag),ΔI 814 (mag),Reff (arcsec),q,P.A. (deg),Classification,DEC_y
0,SDSSJ1422+0203,250704,215.57813,2.054526,534,51997,481,0534-51997-481,0.1104,0.5176,172±9,16.39,0.07,2.05,0.72,175.0,E-S-C,2.05453


In [140]:
zoo_with_mass[zoo_with_mass.GAMA_ID == 250704]

,Unnamed: 0,GAMA_ID,RA,DEC,LENS_SCORE,STELLAR_MASS,Z
8004,8004,250704,215.5782,2.05467,0.00086,1.128000e+11,0.1104


In [142]:
zoo_121519[zoo_121519.GAMA_ID == 250704]

,Unnamed: 0,GAMA_ID,RA,DEC,STELLAR_MASS,Z,NONE,RING,LENS,IRREGULAR,OTHER,DUST_LANE,OVERLAPPING


# S4TM identified one candidate in the GAMA/KiDS fields that was given a very poor score by GZ and was not identified by the other two methods.

# BELLS
### Based on BOSS, SDSS DR8 (according to paper), my data pulled form DR9 because there were no matches in DR8

In [144]:
bells_data = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Publications/BELLS_data.csv')
bells_data

,System Name,ra,dec,Plate-MJD-Fiber,zL,zS,i-band Magnitude,Reff,σBOSS,Sample
0,SDSS J015107.37+004909.0,27.780715,0.819158,3606-55182-0679,0.5171,1.3636,19.81 ± 0.05,0.89 ± 0.21,219 ± 39,cmass
1,SDSS J075754.10+431353.8,119.475400,43.231607,3676-55186-0063,0.5146,1.2165,19.13 ± 0.04,2.51 ± 0.55,...,cmass
2,SDSS J074734.75+444859.3,116.894800,44.816484,3676-55186-0581,0.4366,0.8966,18.84 ± 0.03,2.87 ± 0.50,281 ± 52,cmass
3,SDSS J074724.12+505537.5,116.850520,50.927077,3677-55205-0551,0.4384,0.8983,18.92 ± 0.03,1.24 ± 0.22,328 ± 60,cmass
4,SDSS J080105.30+472749.6,120.272080,47.463764,3684-55246-0301,0.4831,1.5181,19.87 ± 0.05,0.57 ± 0.17,98 ± 24,cmass
5,SDSS J083049.73+511631.8,127.707220,51.275513,3695-55212-0142,0.5301,1.3317,19.50 ± 0.04,1.10 ± 0.22,268 ± 36,cmass
6,SDSS J084121.74+501720.4,130.340590,50.289004,3697-55290-0087,0.5542,1.2881,19.50 ± 0.03,0.79 ± 0.17,176 ± 25,cmass
7,SDSS J083727.02+493703.7,129.362590,49.617696,3697-55290-0255,0.5513,1.1896,19.50 ± 0.03,1.24 ± 0.27,208 ± 34,cmass
8,SDSS J084054.19+505153.1,130.225780,50.864760,3697-55290-0921,0.5534,1.4348,19.81 ± 0.03,0.65 ± 0.17,206 ± 32,cmass
9,SDSS J082130.66+373330.7,125.377770,37.558518,3760-55268-0699,0.5056,0.9705,19.24 ± 0.02,0.93 ± 0.12,203 ± 23,cmass


In [146]:
# compare with gama data
bells_in_G09 = bells_data[(bells_data.ra >= 129.0) & (bells_data.ra <= 141.0) & (bells_data.dec >= -2) & (bells_data.dec <= 3)]
bells_in_G12 = bells_data[(bells_data.ra >= 174.0) & (bells_data.ra <= 186.0) & (bells_data.dec >= -3) & (bells_data.dec <= 2)]                   
bells_in_G15 = bells_data[(bells_data.ra >= 211.5) & (bells_data.ra <= 223.5) & (bells_data.dec >= -2) & (bells_data.dec <= 3)]

In [147]:
bells_in_G09

,System Name,ra,dec,Plate-MJD-Fiber,zL,zS,i-band Magnitude,Reff,σBOSS,Sample
10,SDSS J091516.60−005500.6,138.81917,-0.916841,3766-55213-0675,0.4022,1.1705,18.46 ± 0.02,1.98 ± 0.28,221 ± 16,cmass


In [148]:
bells_in_G12

,System Name,ra,dec,Plate-MJD-Fiber,zL,zS,i-band Magnitude,Reff,σBOSS,Sample
11,SDSS J122113.28−022037.8,185.30535,-2.343838,3777-55210-0075,0.4061,1.0108,19.07 ± 0.02,0.64 ± 0.09,240 ± 24,cmass
18,SDSS J115944.63−000728.2,179.93596,-0.124500,3843-55278-0069,0.5793†,1.3457,19.63 ± 0.03,0.99 ± 0.22,165 ± 41,cmass
19,SDSS J121504.44+004726.0,183.76849,0.790569,3846-55327-0559,0.6423,1.2970,19.27 ± 0.03,1.42 ± 0.21,262 ± 45,cmass


In [253]:
#bells_in_gama = pd.concat([bells_in_G09,bells_in_G12])
#bells_in_gama = bells_in_gama.rename(columns = {'ra':'RA','dec':'DEC'})
#bells_in_gama = bells_in_gama.reset_index()
#bells_in_gama = bells_in_gama.drop(columns = ['index'])
#bells_in_gama.to_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/bells_gama.csv')
bells_in_gama

,System Name,RA,DEC,Plate-MJD-Fiber,zL,zS,i-band Magnitude,Reff,σBOSS,Sample
0,SDSS J091516.60−005500.6,138.81917,-0.916841,3766-55213-0675,0.4022,1.1705,18.46 ± 0.02,1.98 ± 0.28,221 ± 16,cmass
1,SDSS J122113.28−022037.8,185.30535,-2.343838,3777-55210-0075,0.4061,1.0108,19.07 ± 0.02,0.64 ± 0.09,240 ± 24,cmass
2,SDSS J115944.63−000728.2,179.93596,-0.124500,3843-55278-0069,0.5793†,1.3457,19.63 ± 0.03,0.99 ± 0.22,165 ± 41,cmass
3,SDSS J121504.44+004726.0,183.76849,0.790569,3846-55327-0559,0.6423,1.2970,19.27 ± 0.03,1.42 ± 0.21,262 ± 45,cmass


In [203]:
#bells_gama = pd.merge(bells_in_gama, gama_data, how = 'left', on = 'DEC')
#bells_gama
bells_gama = pd.DataFrame(columns = ['GAMA_ID', 'SDSS_ID', 'RA', 'DEC'], index = range(len(bells_in_gama)))
for i in range(len(bells_in_gama.DEC)):
    Rlim = 2./3600.
    for j in range(len(gama_data)):
        R = np.sqrt(np.square(bells_in_gama.DEC[i]-gama_data.DEC[j])+np.square(bells_in_gama.RA[i]-gama_data.RA[j]))
        if R <= Rlim:
            Rlim = R
            bells_gama.GAMA_ID[i] = gama_data.GAMA_ID[j]
            bells_gama.SDSS_ID[i] = bells_in_gama.SDSS_ID[i]
            bells_gama.RA[i] = gama_data.RA[j]
            bells_gama.DEC[i] = gama_data.DEC[j]
            print(bells_gama)

# There are four BELLS candidates in the GAMA regions, but none of them match the GAMA DR3 database, and therefore have no match in the GAMA/KiDS lens searches.

# DECALS

In [ ]:
# Opening GAMA data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/DECALS_survey-dr7-dr7Q.fits')  # open a FITS file
hdul.verify('fix')
decals = hdul[1].data  # assume the first extension is a table
#print(decals.columns)

#decals_id = decals['OBJID']
#decals_ra = gama['RA']
#decals_dec = gama['DEC']

#decals_list = list(zip(decals_id,decals_ra,decals_dec))

#decals_data = pd.DataFrame(decals_list, columns = ['DECALS_ID','RA','DEC'])
#decals_data

In [227]:
decals.SMJD

array([52203, 52203, 52251, ..., 52203, 52143, 52991], dtype=int32)

In [210]:
# compare with gama data
decals_in_G09 = decals_data[(decals_data.RA >= 129.0) & (decals_data.RA <= 141.0) & (decals_data.DEC >= -2) & (decals_data.DEC <= 3)]
decals_in_G12 = decals_data[(decals_data.RA >= 174.0) & (decals_data.RA <= 186.0) & (decals_data.DEC >= -3) & (decals_data.DEC <= 2)]                   
decals_in_G15 = decals_data[(decals_data.RA >= 211.5) & (decals_data.RA <= 223.5) & (decals_data.DEC >= -2) & (decals_data.DEC <= 3)]

In [228]:
decals_A = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/decals_A_candidates.csv')
decals_B = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/decals_B_candidates.csv')
decals_C = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/decals_C_candidates.csv')

### DECALS A candidates... one found in G15

In [233]:
decals_A_in_G09 = decals_A[(decals_A.RA >= 129.0) & (decals_A.RA <= 141.0) & (decals_A.DEC >= -2) & (decals_A.DEC <= 3)]
decals_A_in_G12 = decals_A[(decals_A.RA >= 174.0) & (decals_A.RA <= 186.0) & (decals_A.DEC >= -3) & (decals_A.DEC <= 2)]                   
decals_A_in_G15 = decals_A[(decals_A.RA >= 211.5) & (decals_A.RA <= 223.5) & (decals_A.DEC >= -2) & (decals_A.DEC <= 3)]

In [255]:
#decals_A_gama = decals_A_in_G15
#decals_A_gama['Rank']='A'
decals_A_gama

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey,Rank
51,DESI-215.2654+00.3719,215.2654,0.3719,DEV,21.32,20.05,18.73,0.974,NaN,NaN,A


### DECALS B candidates... three found in GAMA eq regions

In [238]:
decals_B_in_G09 = decals_B[(decals_B.RA >= 129.0) & (decals_B.RA <= 141.0) & (decals_B.DEC >= -2) & (decals_B.DEC <= 3)]
decals_B_in_G12 = decals_B[(decals_B.RA >= 174.0) & (decals_B.RA <= 186.0) & (decals_B.DEC >= -3) & (decals_B.DEC <= 2)]                   
decals_B_in_G15 = decals_B[(decals_B.RA >= 211.5) & (decals_B.RA <= 223.5) & (decals_B.DEC >= -2) & (decals_B.DEC <= 3)]

In [239]:
decals_B_in_G09

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey
68,DESI-131.3607+00.0361,131.3607,0.0361,DEV,24.15,21.79,19.7,0.8,NaN,NaN


In [241]:
decals_B_in_G15

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey
98,DESI-219.0374-01.3295,219.0374,-1.3295,DEV,20.10,18.66,17.6,0.847,NaN,NaN
99,DESI-219.9228+00.5073,219.9228,0.5073,DEV,18.67,17.67,17.0,0.837,0.1377,SDSS


In [257]:
#decals_B_gama = pd.concat([decals_B_in_G09, decals_B_in_G15])
#decals_B_gama['Rank'] = 'B'
decals_B_gama

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey,Rank
68,DESI-131.3607+00.0361,131.3607,0.0361,DEV,24.15,21.79,19.7,0.800,NaN,NaN,B
98,DESI-219.0374-01.3295,219.0374,-1.3295,DEV,20.10,18.66,17.6,0.847,NaN,NaN,B
99,DESI-219.9228+00.5073,219.9228,0.5073,DEV,18.67,17.67,17.0,0.837,0.1377,SDSS,B


### DECALS C... found two in GAMA eq regions.

In [242]:
decals_C_in_G09 = decals_C[(decals_C.RA >= 129.0) & (decals_C.RA <= 141.0) & (decals_C.DEC >= -2) & (decals_C.DEC <= 3)]
decals_C_in_G12 = decals_C[(decals_C.RA >= 174.0) & (decals_C.RA <= 186.0) & (decals_C.DEC >= -3) & (decals_C.DEC <= 2)]                   
decals_C_in_G15 = decals_C[(decals_C.RA >= 211.5) & (decals_C.RA <= 223.5) & (decals_C.DEC >= -2) & (decals_C.DEC <= 3)]

In [243]:
decals_C_in_G09

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey
122,DESI-138.6664-00.0821,138.6664,-0.0821,22.79,21.03,19.69,0.953,NaN,NaN,NaN


In [244]:
decals_C_in_G12

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey
138,DESI-180.0490-00.4182,180.049,-0.4182,COMP,23.41,21.54,19.96,0.971,NaN,NaN


In [259]:
#decals_C_gama = pd.concat([decals_C_in_G09, decals_C_in_G12])
#decals_C_gama['Rank'] = 'C'
decals_C_gama

,Name,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey,Rank
122,DESI-138.6664-00.0821,138.6664,-0.0821,22.79,21.03,19.69,0.953,NaN,NaN,NaN,C
138,DESI-180.0490-00.4182,180.0490,-0.4182,COMP,23.41,21.54,19.960,0.971,NaN,NaN,C


### Combine A, B and C

In [281]:
#decals_gama = pd.concat([decals_A_gama,decals_B_gama,decals_C_gama])
#decals_gama.to_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/decals_gama.csv')
#decals_gama = decals_gama.reset_index()
#decals_gama = decals_gama.drop(columns = ['index'])
#decals_gama = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/decals_gama.csv')
#decals_gama = decals_gama.rename(columns = {'Name':'DESI_ID'})
decals_gama

,Unnamed: 0,DESI_ID,RA,DEC,Type,mag g,mag r,mag z,Probability,z,Survey,Rank
0,51,DESI-215.2654+00.3719,215.2654,0.3719,DEV,21.32,20.05,18.730,0.974,NaN,NaN,A
1,68,DESI-131.3607+00.0361,131.3607,0.0361,DEV,24.15,21.79,19.700,0.800,NaN,NaN,B
2,98,DESI-219.0374-01.3295,219.0374,-1.3295,DEV,20.10,18.66,17.600,0.847,NaN,NaN,B
3,99,DESI-219.9228+00.5073,219.9228,0.5073,DEV,18.67,17.67,17.000,0.837,0.1377,SDSS,B
4,122,DESI-138.6664-00.0821,138.6664,-0.0821,22.79,21.03,19.69,0.953,NaN,NaN,NaN,C
5,138,DESI-180.0490-00.4182,180.0490,-0.4182,COMP,23.41,21.54,19.960,0.971,NaN,NaN,C


In [271]:
decals_in_gama = pd.DataFrame(columns = ['GAMA_ID', 'DESI_ID', 'RA', 'DEC'], index = range(len(decals_gama)))
for i in range(len(decals_gama.DEC)):
    Rlim = 2./3600.
    for j in range(len(gama_data)):
        R = np.sqrt(np.square(decals_gama.DEC[i]-gama_data.DEC[j])+np.square(decals_gama.RA[i]-gama_data.RA[j]))
        if R <= Rlim:
            Rlim = R
            decals_in_gama.GAMA_ID[i] = gama_data.GAMA_ID[j]
            decals_in_gama.DESI_ID[i] = decals_gama.Name[i]
            decals_in_gama.RA[i] = gama_data.RA[j]
            decals_in_gama.DEC[i] = gama_data.DEC[j]
            print(decals_in_gama)

  GAMA_ID                DESI_ID       RA      DEC
0     NaN                    NaN      NaN      NaN
1     NaN                    NaN      NaN      NaN
2  493010  DESI-219.0374-01.3295  219.037 -1.32946
3     NaN                    NaN      NaN      NaN
4     NaN                    NaN      NaN      NaN
5     NaN                    NaN      NaN      NaN
  GAMA_ID                DESI_ID       RA      DEC
0     NaN                    NaN      NaN      NaN
1     NaN                    NaN      NaN      NaN
2  493010  DESI-219.0374-01.3295  219.037 -1.32946
3     NaN                    NaN      NaN      NaN
4     NaN                    NaN      NaN      NaN
5     NaN                    NaN      NaN      NaN
  GAMA_ID                DESI_ID       RA      DEC
0     NaN                    NaN      NaN      NaN
1     NaN                    NaN      NaN      NaN
2  493010  DESI-219.0374-01.3295  219.037 -1.32946
3     NaN                    NaN      NaN      NaN
4     NaN                    Na

In [272]:
decals_in_gama

,GAMA_ID,DESI_ID,RA,DEC
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,493010,DESI-219.0374-01.3295,219.037,-1.32946
3,93310,DESI-219.9228+00.5073,219.923,0.50731
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN


In [288]:
#decals_in_gama = pd.merge(decals_gama, decals_in_gama, on = 'DESI_ID', how = 'left')
#decals_in_gama = decals_in_gama[['DESI_ID', 'GAMA_ID', 'Rank', 'RA_x', 'DEC_x', 'RA_y',
  #     'DEC_y', 'Type', 'mag g', 'mag r',
   #    'mag z', 'Probability', 'z', 'Survey', 'Unnamed: 0']]
#decals_in_gama.to_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/decals_in_gama.csv')
decals_in_gama

,DESI_ID,GAMA_ID,Rank,RA_x,DEC_x,RA_y,DEC_y,Type,mag g,mag r,mag z,Probability,z,Survey,Unnamed: 0
0,DESI-215.2654+00.3719,NaN,A,215.2654,0.3719,NaN,NaN,DEV,21.32,20.05,18.730,0.974,NaN,NaN,51
1,DESI-131.3607+00.0361,NaN,B,131.3607,0.0361,NaN,NaN,DEV,24.15,21.79,19.700,0.800,NaN,NaN,68
2,DESI-219.0374-01.3295,493010,B,219.0374,-1.3295,219.037,-1.32946,DEV,20.10,18.66,17.600,0.847,NaN,NaN,98
3,DESI-219.9228+00.5073,93310,B,219.9228,0.5073,219.923,0.50731,DEV,18.67,17.67,17.000,0.837,0.1377,SDSS,99
4,DESI-138.6664-00.0821,NaN,C,138.6664,-0.0821,NaN,NaN,22.79,21.03,19.69,0.953,NaN,NaN,NaN,122
5,DESI-180.0490-00.4182,NaN,C,180.0490,-0.4182,NaN,NaN,COMP,23.41,21.54,19.960,0.971,NaN,NaN,138


In [291]:
pd.merge(decals_in_gama, zoo_121519, on = 'GAMA_ID')

,DESI_ID,GAMA_ID,Rank,RA_x,DEC_x,RA_y,DEC_y,Type,mag g,mag r,...,DEC,STELLAR_MASS,Z,NONE,RING,LENS,IRREGULAR,OTHER,DUST_LANE,OVERLAPPING
0,DESI-219.9228+00.5073,93310,B,219.9228,0.5073,219.923,0.50731,DEV,18.67,17.67,...,0.50731,3.094000e+10,0.1377,0.424861,0.0,0.575139,0.0,0.0,0.0,0.0


# Decals found one match with zoo of rank B.

# DES (Jacobs et al. 2019)

In [ ]:
#des_data = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/Jacobs_2019_DES.csv')
#des_data = des_data.rename(columns = {'dec':'DEC'})
des_data

In [307]:
des_data_in_G09 = des_data[(des_data.RA >= 129.0) & (des_data.RA <= 141.0) & (des_data.DEC >= -2) & (des_data.DEC <= 3)]
des_data_in_G12 = des_data[(des_data.RA >= 174.0) & (des_data.RA <= 186.0) & (des_data.DEC >= -3) & (des_data.DEC <= 2)]                   
des_data_in_G15 = des_data[(des_data.RA >= 211.5) & (des_data.RA <= 223.5) & (des_data.DEC >= -2) & (des_data.DEC <= 3)]

# DES wasn't anywhere near the GAMA eq regions

---

# Summary of Findings

In [311]:
slacs
# 136604 is a match to Mac and Zoo.

,SDSS_ID,z lens,z src,GAMA_ID,RA,DEC
0,SDSSJ0912+0029,0.164,0.324,216398,138.022,0.48366
1,SDSSJ1143−0144,0.106,0.402,136604,175.873,-1.74167
2,SDSSJ1436−0000,0.285,0.805,NaN,NaN,NaN
3,SDSSJ1451−0239,0.125,0.520,NaN,NaN,NaN


In [316]:
s4tm_gama
# 250704 is given very poor score in Zoo (didn't make my cuts)

,Target,GAMA_ID,RA,DEC_x,plate,mjd,fiberid,Plate-MJD-Fiber,z L,z S,σSDSS (km s −1),I 814 (mag),ΔI 814 (mag),Reff (arcsec),q,P.A. (deg),Classification,DEC_y
0,SDSSJ1422+0203,250704,215.57813,2.054526,534,51997,481,0534-51997-481,0.1104,0.5176,172±9,16.39,0.07,2.05,0.72,175.0,E-S-C,2.05453


In [314]:
bells_in_gama
# No match in GAMA DR3 database

,System Name,RA,DEC,Plate-MJD-Fiber,zL,zS,i-band Magnitude,Reff,σBOSS,Sample
0,SDSS J091516.60−005500.6,138.81917,-0.916841,3766-55213-0675,0.4022,1.1705,18.46 ± 0.02,1.98 ± 0.28,221 ± 16,cmass
1,SDSS J122113.28−022037.8,185.30535,-2.343838,3777-55210-0075,0.4061,1.0108,19.07 ± 0.02,0.64 ± 0.09,240 ± 24,cmass
2,SDSS J115944.63−000728.2,179.93596,-0.124500,3843-55278-0069,0.5793†,1.3457,19.63 ± 0.03,0.99 ± 0.22,165 ± 41,cmass
3,SDSS J121504.44+004726.0,183.76849,0.790569,3846-55327-0559,0.6423,1.2970,19.27 ± 0.03,1.42 ± 0.21,262 ± 45,cmass


In [315]:
decals_in_gama
# 93310 matches Zoo final cut. 493010 does not match.

,DESI_ID,GAMA_ID,Rank,RA_x,DEC_x,RA_y,DEC_y,Type,mag g,mag r,mag z,Probability,z,Survey,Unnamed: 0
0,DESI-215.2654+00.3719,NaN,A,215.2654,0.3719,NaN,NaN,DEV,21.32,20.05,18.730,0.974,NaN,NaN,51
1,DESI-131.3607+00.0361,NaN,B,131.3607,0.0361,NaN,NaN,DEV,24.15,21.79,19.700,0.800,NaN,NaN,68
2,DESI-219.0374-01.3295,493010,B,219.0374,-1.3295,219.037,-1.32946,DEV,20.10,18.66,17.600,0.847,NaN,NaN,98
3,DESI-219.9228+00.5073,93310,B,219.9228,0.5073,219.923,0.50731,DEV,18.67,17.67,17.000,0.837,0.1377,SDSS,99
4,DESI-138.6664-00.0821,NaN,C,138.6664,-0.0821,NaN,NaN,22.79,21.03,19.69,0.953,NaN,NaN,NaN,122
5,DESI-180.0490-00.4182,NaN,C,180.0490,-0.4182,NaN,NaN,COMP,23.41,21.54,19.960,0.971,NaN,NaN,138
